## Third Movement: 

In [1]:
# Import required libraries
import networkx as nx
import numpy as np
from scipy.special import factorial
from datetime import datetime
import logging
from scipy.stats import geom
import sys
import os

In [2]:
import os
experiment_log_file = 'experiment_4.log'
log_path = os.path.join(os.getcwd(), experiment_log_file)

if os.path.exists(log_path):
    os.remove(log_path)

logger = logging.getLogger()
handler = logging.FileHandler(log_path, mode='w')
handler.setFormatter(logging.Formatter('%(message)s'))
logger.addHandler(handler)
logger.setLevel(logging.INFO)

logger.info(f'[1] {experiment_log_file}')
logger.info(f'[1] "{datetime.now().strftime("%a %b %d %H:%M:%S %Y")}"')

In [3]:
%run attack_graph_MIR100.ipynb

### Use Third Distribution

In [4]:
attack_rate_list = [0]   
defense_rate_list = [1,2,3]  

In [5]:
def random_steps(route, attack_rate=None, defense_rate=None, graph=None):
    # Get hardness values from the graph
    hardness = []
    for i in range(len(route) - 1):
        u, v = route[i], route[i+1]
        edges_dict = graph[u][v]

        if isinstance(edges_dict, float):
            total_prob = edges_dict
        elif hasattr(edges_dict, 'items'):
            total_prob = 0.0
            for edge_key, attr_dict in edges_dict.items():
                if isinstance(attr_dict, float):
                    total_prob += attr_dict
                else:
                    p = float(attr_dict.get('prob', 1.0))
                    total_prob += p
        else:
            p = float(edges_dict.get('prob', 1.0))
            total_prob = p

        hardness.append(total_prob)

    hardness = np.array(hardness)
    hardness = np.nan_to_num(hardness, nan=1.0)

    # Calculate geometric mean for attack rate
    def geometric_mean(x):
        positive_values = x[x > 0]
        return np.exp(np.mean(np.log(positive_values)))

    if attack_rate is None:
        attack_rate = 1 / geometric_mean(hardness)

    # Calculate probability for geometric distribution
    if attack_rate == 0 and defense_rate == 0:
        # When both rates are 0, return uniform distribution
        return np.ones(len(route)) / len(route)
        
    prob = attack_rate / (attack_rate + defense_rate)
    
    # Generate geometric distribution
    x = np.arange(len(route))
    
    if attack_rate == 0:
        # Replicate R's dgeom behavior when prob = 0
        # The first value should be 0, rest should decrease with defense_rate
        pdf = np.exp(-defense_rate * x)
        pdf[0] = 0  # First value should be 0 when attack_rate = 0
    else:
        pdf = geom.pmf(x, prob)
    
    # Handle any remaining NaN values
    if not np.any(np.isfinite(pdf)):
        pdf = np.ones_like(pdf)
    
    # Normalize
    pdf = pdf / np.sum(pdf)
    
    return pdf

In [6]:
# %run ctr-core_simple.ipynb
%run ctr-core_tests.ipynb
main()


After merging targets:
Nodes: [1, 5, 15, 11, 3, 6, 8, 4, 7, 2, 9, 10, 0, 'c(12,13,14,16)']
Edges with weights:
1 -> 5 (key=0) : 2.1958405355640576
5 -> 15 (key=0) : 2.1958405355640576
15 -> c(12,13,14,16) (key=0) : 0.7489220813074156
15 -> c(12,13,14,16) (key=1) : 0.7489220813074156
15 -> c(12,13,14,16) (key=2) : 0.7489220813074156
11 -> c(12,13,14,16) (key=0) : 1.064439873679208
11 -> c(12,13,14,16) (key=1) : 0.7489220813074156
11 -> c(12,13,14,16) (key=2) : 0.0
3 -> 6 (key=0) : 1.064439873679208
3 -> 8 (key=0) : 0.7489220813074156
6 -> 8 (key=0) : 0.0
8 -> c(12,13,14,16) (key=0) : 0.0
8 -> c(12,13,14,16) (key=1) : 0.7489220813074156
8 -> 10 (key=0) : 0.0
4 -> 7 (key=0) : 0.7489220813074156
7 -> c(12,13,14,16) (key=0) : 0.7489220813074156
7 -> c(12,13,14,16) (key=1) : 0.7489220813074156
7 -> 10 (key=0) : 0.7489220813074156
2 -> c(12,13,14,16) (key=0) : 1.064439873679208
2 -> 9 (key=0) : 0.7489220813074156
2 -> 10 (key=0) : 1.064439873679208
2 -> 11 (key=0) : 0.7489220813074156
9 -> c

In [7]:
with open(experiment_log_file, 'r') as f:
    print(f.read())

[1] experiment_4.log
[1] "Wed Jan 08 16:45:53 2025"

++++++++++++++++++++++++++++++++

The virtual target nodeID is c(12,13,14,16)

attack rate =  0 , defense rate =  1 

	equilibrium for multiobjective security game (MOSG)

optimal defense strategy:
         prob.
10 0.000000e+00
11 1.777941e-01
15 2.343534e-01
5 0.000000e+00
6 0.000000e+00
7 1.777941e-01
8 2.322644e-01
9 1.777941e-01

worst case attack strategies per goal:
          1
1 0.0000000
2 0.1962398
3 0.0000000
4 0.0000000
5 0.0000000
6 0.1917171
7 0.1917172
8 0.0000000
9 0.2101629
10 0.0000000
11 0.2101629
[1] 0.087

Defender can keep attacker success below: 0.087
Attacker can guarantee success probability of: 0.087

++++++++++++++++++++++++++++++++

The virtual target nodeID is c(12,13,14,16)

attack rate =  0 , defense rate =  2 

	equilibrium for multiobjective security game (MOSG)

optimal defense strategy:
         prob.
10 0.000000e+00
11 5.489772e-01
15 1.190043e-01
5 0.000000e+00
6 0.000000e+00
7 1.065071e-01
8 1.19